# TV Script Generation
In this project, I will generate my own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  I will use parts of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network I will build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
I will be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)

text = text[81:]

## Data exploration

In [2]:
view_sentence_range = (0, 10)

import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.251908396946565
Number of lines: 4258
Average number of words in each line: 11.50164396430249

The sentences 0 to 10:

Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.



## Implement Preprocessing Functions

- Lookup Table
- Tokenize Punctuation

### Lookup Table
Transform the words to ids.  In this function, two dictionaries are created:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

In [3]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):

    # Lookup table
    vocab = set(text)
    vocab_to_int = {c: i for i, c in enumerate(vocab)}
    int_to_vocab = dict(enumerate(vocab))
    
    return vocab_to_int, int_to_vocab

tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
This will split the script into a word array using spaces as delimiters. However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implementation of the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||". A dictionary for the following symbols where the symbol is the key and value is the token is created:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word.

In [4]:
def token_lookup():

    # key : value
    token_lookup = {'.': '||Period||', ',':'||comma||', '"':'QuotationMark', ';':'Semicolon', '!':'ExclamationMark',
                    '?':'QuestionMark', '(':'LeftParentheses', ')':'RightParentheses', '--':'||Dash||', 
                    '\n':'||Return||'}
    
    return token_lookup

tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it

In [5]:
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
First checkpoint.

In [6]:
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
The following functions are necessary to build the RNN:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [8]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('GPU found! Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.0
GPU found! Default GPU Device: /gpu:0


### Input
Implementation of the `get_inputs()` function to create TF Placeholders for the Neural Network.  

- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

In [9]:
def get_inputs():

    # Create placeholders
    inputs_ = tf.placeholder(tf.int32, [None, None], name='input')
    labels_ = tf.placeholder(tf.int32, [None, None], name='labels')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    return inputs_, labels_, learning_rate

tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize

In [10]:
def get_init_cell(batch_size, rnn_size):

    # Set layers number and dropout parameter
    num_layers = 2
    keep_prob = 0.7

    Cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.BasicLSTMCell(rnn_size) for _ in range(num_layers)])
    InitialState = Cell.zero_state(batch_size, tf.float32)
    InitialState = tf.identity(InitialState, name='initial_state')

    return Cell, InitialState

tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.

In [11]:
def get_embed(input_data, vocab_size, embed_dim):

    # Get embedding
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim),-1,1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    
    return embed

tests.test_get_embed(get_embed)

Tests Passed


### Build RNN

In [12]:
def build_rnn(cell, inputs):

    # Build the dynamic rnn
    Outputs, FinalState = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    FinalState = tf.identity(FinalState, name='final_state')
    return Outputs, FinalState

tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network

In [13]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):

    # RNN pipeline
    inputs = get_embed(input_data, vocab_size, embed_dim)
    output, FinalState = build_rnn(cell, inputs)
    Logits = tf.contrib.layers.fully_connected(output, vocab_size, activation_fn = None)
    
    return Logits, FinalState

tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [14]:
def get_batches(int_text, batch_size, seq_length):

    # Get batches pipeline
    batch_s = batch_size * seq_length
    n_batches = int(len(int_text)/ batch_s)
    
    input_data = np.array(int_text[:n_batches * batch_s])
    targets_data = np.array(int_text[1:n_batches * batch_s + 1])
    targets_data[-1] = input_data[0]
    
    input_batches = np.split(input_data.reshape(batch_size, -1), n_batches, 1)
    targets_batches = np.split(targets_data.reshape(batch_size, -1), n_batches, 1)
    
    return np.array(list(zip(input_batches, targets_batches)))

# Check the function
print(get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2))
tests.test_get_batches(get_batches)

[[[[ 1  2]
   [ 7  8]
   [13 14]]

  [[ 2  3]
   [ 8  9]
   [14 15]]]


 [[[ 3  4]
   [ 9 10]
   [15 16]]

  [[ 4  5]
   [10 11]
   [16 17]]]


 [[[ 5  6]
   [11 12]
   [17 18]]

  [[ 6  7]
   [12 13]
   [18  1]]]]
Tests Passed


## Neural Network Training
### Hyperparameters

- `num_epochs` - the number of epochs.
- `batch_size` - the batch size.
- `rnn_size` - the size of the RNNs.
- `embed_dim` - the size of the embedding.
- `seq_length` - the length of sequence.
- `learning_rate` - the learning rate.
- `show_every_n_batches` - the number of batches the neural network should print progress.

In [15]:
# Number of Epochs
num_epochs = 65
# Batch Size
batch_size = 256
# RNN Size
rnn_size = 1024
# Embedding Dimension Size
embed_dim = 250
# Sequence Length
seq_length = 10
# Learning Rate
learning_rate = 0.008
# Show stats for every n number of batches
show_every_n_batches = 10

save_dir = './save'

### Build the Graph
Build the graph using the implemented neural network.

In [16]:
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.

In [17]:
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/26   train_loss = 8.822
Epoch   0 Batch   10/26   train_loss = 7.474
Epoch   0 Batch   20/26   train_loss = 6.543
Epoch   1 Batch    4/26   train_loss = 6.061
Epoch   1 Batch   14/26   train_loss = 6.044
Epoch   1 Batch   24/26   train_loss = 5.836
Epoch   2 Batch    8/26   train_loss = 5.703
Epoch   2 Batch   18/26   train_loss = 5.601
Epoch   3 Batch    2/26   train_loss = 5.514
Epoch   3 Batch   12/26   train_loss = 5.375
Epoch   3 Batch   22/26   train_loss = 5.256
Epoch   4 Batch    6/26   train_loss = 5.331
Epoch   4 Batch   16/26   train_loss = 5.094
Epoch   5 Batch    0/26   train_loss = 5.147
Epoch   5 Batch   10/26   train_loss = 4.944
Epoch   5 Batch   20/26   train_loss = 5.067
Epoch   6 Batch    4/26   train_loss = 4.938
Epoch   6 Batch   14/26   train_loss = 5.047
Epoch   6 Batch   24/26   train_loss = 4.920
Epoch   7 Batch    8/26   train_loss = 4.911
Epoch   7 Batch   18/26   train_loss = 4.822
Epoch   8 Batch    2/26   train_loss = 4.740
Epoch   8 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [18]:
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [19]:
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

In [20]:
def get_tensors(loaded_graph):

    # Get tensors functions
    InputTensor = loaded_graph.get_tensor_by_name("input:0")
    InitialStateTensor = loaded_graph.get_tensor_by_name("initial_state:0")
    FinalStateTensor = loaded_graph.get_tensor_by_name("final_state:0")
    ProbsTensor = loaded_graph.get_tensor_by_name("probs:0")
    
    return InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor

tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implementation of the `pick_word()` function to select the next word using `probabilities`.

In [21]:
def pick_word(probabilities, int_to_vocab):

    # Get the probability for next word selection
    prob = np.random.choice(list(int_to_vocab.values()), 1, p=probabilities)[0]
    
    return prob

tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script.  Set `gen_length` to the length of TV script you want to generate.

In [23]:
gen_length = 500

# Choose a character by setting the prime word 
# Characters: homer_simpson, moe_szyslak, or barney_gumble
prime_word = 'barney_gumble'

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

barney_gumble:(in moe's face) lord palmerston!


moe_szyslak: i feel so about what?(then, homer) i forgot to endorse this loser. they're bashir with one thing are you, have how i no with some song you said everything(laughs) i was the thing to get my au gay drive here to drive the cleveland tavern.
lenny_leonard: what you are not one of your monkeyshines.
lenny_leonard: that's the foibles to have to moe.
homer_simpson:(low, on glass) when the crony.
homer_simpson: i can't close?
homer_simpson: now about it.
moe_szyslak: good good who, i'll get renovatin'.
bart_simpson: how 7-year-old_brockman: they a beer!
moe_szyslak:(calling something) you're the phone) can i just said that you're good.
moe_szyslak: it was far the money, i'll check...
moe_szyslak: well moe.(low, to the super) i want you to say to you!
lisa_simpson: that comes to go! i'm supposed on a shotgun.
moe_szyslak: hey, can i look like that.
homer_simpson: so you said you're in the best musical girls?
homer_simpson:(flailing) 

## Discussion: The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.